<a href="https://colab.research.google.com/github/coldbilberry/repo-gui/blob/main/%D0%A3%D1%80%D0%BE%D0%BA_2_%D0%94%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82_%D0%BF%D0%BE_%D0%B2%D1%8B%D0%B7%D0%B2%D0%B0%D0%BD%D0%BD%D1%8B%D0%BC_%D0%BF%D0%BE%D1%82%D0%B5%D0%BD%D1%86%D0%B8%D0%B0%D0%BB%D0%B0%D0%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.6 MB/s eta 0:00:00


In [2]:
from google.colab import files

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import KBinsDiscretizer

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import r2_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import learning_curve

from sklearn.ensemble import RandomForestClassifier

from catboost import CatBoostClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings('ignore')

In [3]:
def get_metrics(y_test, y_pred, y_score, name):
    df_metrics = pd.DataFrame()
    df_metrics['model'] = [name]
    df_metrics['Accuracy'] = accuracy_score(y_test, y_pred)
    df_metrics['R2'] = r2_score(y_test, y_pred)

    return df_metrics

In [4]:
uploaded = files.upload()

KeyboardInterrupt: 

In [ ]:
!unzip relative+location+of+ct+slices+on+axial+axis.zip

In [ ]:
df = pd.read_csv('slice_localization_data.csv')

In [ ]:
df

In [ ]:
df.info()

In [ ]:
print(df.dtypes)

In [ ]:
df.describe().T

In [ ]:
df['reference'].hist(bins=50, edgecolor='black')

In [ ]:
df['patientId'].value_counts() # распределение слайсов КТ по пациентам

In [ ]:
df['patientId'].hist(bins=96, edgecolor='black')

In [ ]:
df_0 = df[df['patientId'] == 46]

plt.figure(figsize=(10, 6))
sns.histplot(df_0['reference'], bins=50, kde=False, color='blue', edgecolor='black')
plt.title('Гистограмма значений для patient 0')
plt.xlabel('reference')
plt.ylabel('Частота')
plt.show()

In [ ]:
df_0 = df[df['patientId'] == 63]

plt.figure(figsize=(10, 6))
sns.histplot(df_0['reference'], bins=50, kde=False, color='blue', edgecolor='black')
plt.title('Гистограмма значений для patient 0')
plt.xlabel('reference')
plt.ylabel('Частота')
plt.show()

In [ ]:
X = df.drop('reference', axis=1)
y = df.reference

In [ ]:
discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')

y = y.values.reshape(-1, 1)

y = discretizer.fit_transform(y.reshape(-1, 1)).astype(int).ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
model_lg = LogisticRegression()

model_lg.fit(X_train, y_train)

In [ ]:
y_pred = model_lg.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy

In [ ]:
r2_score(y_test, y_pred)

In [ ]:
y_score = model_lg.predict_proba(X_test)

metrics_lg_0 = get_metrics(y_test, y_pred, y_score, name='LogisticRegression_0')
metrics_lg_0

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

rf_model.fit(X_train, y_train)

In [ ]:
y_pred = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy

In [ ]:
r2_score(y_test, y_pred )

In [ ]:
y_score = rf_model.predict_proba(X_test)

metrics_rf_0 = get_metrics(y_test, y_pred, y_score, name='RandomForestClassifier_0')
metrics_rf_0

In [ ]:
model_cat = CatBoostClassifier(iterations=100,
                           learning_rate=0.1,
                           depth=6,
                           loss_function='MultiClass',
                           verbose=0)

model_cat.fit(X_train, y_train)

In [ ]:
y_pred = model_cat.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy

In [ ]:
r2_score(y_test, y_pred )

In [ ]:
y_score = model_cat.predict_proba(X_test)

metrics_cat_0 = get_metrics(y_test, y_pred, y_score, name='CatBoostClassifier_0')
metrics_cat_0

In [ ]:
k = 10  # количество соседей
knn_model = KNeighborsClassifier(n_neighbors=k)
knn_model.fit(X_train, y_train)

In [ ]:
y_pred = knn_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy

In [ ]:
r2_score(y_test, y_pred )

In [ ]:
y_score = knn_model.predict_proba(X_test)

In [ ]:
metrics_knn_0 = get_metrics(y_test, y_pred, y_score, name='KNeighborsClassifier_0')
metrics_knn_0

In [ ]:
# удаление patientId
df_clear = df.drop('patientId', axis = 1)

df_clear

In [ ]:
X = df_clear.drop('reference', axis=1)
y = df_clear.reference

In [ ]:
discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')

y = y.values.reshape(-1, 1)

y = discretizer.fit_transform(y.reshape(-1, 1)).astype(int).ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
model_lg = LogisticRegression()

model_lg.fit(X_train, y_train)

In [ ]:
y_pred = model_lg.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy

In [ ]:
r2_score(y_test, y_pred)

In [ ]:
y_score = model_lg.predict_proba(X_test)

metrics_lg = get_metrics(y_test, y_pred, y_score, name='LogisticRegression')
metrics_lg

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

rf_model.fit(X_train, y_train)

In [5]:
y_pred = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy

NameError: name 'rf_model' is not defined

In [ ]:
r2_score(y_test, y_pred )

In [ ]:
model_cat = CatBoostClassifier(iterations=100,
                           learning_rate=0.1,
                           depth=6,
                           loss_function='MultiClass',
                           verbose=0)

model_cat.fit(X_train, y_train)

In [ ]:
y_pred = model_cat.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy

In [ ]:
r2_score(y_test, y_pred )

In [ ]:
y_score = model_cat.predict_proba(X_test)

metrics_cat = get_metrics(y_test, y_pred, y_score, name='CatBoostClassifier')
metrics_cat

In [ ]:
k = 10  # количество соседей
knn_model = KNeighborsClassifier(n_neighbors=k)
knn_model.fit(X_train, y_train)

In [ ]:
y_pred = knn_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy

In [ ]:
r2_score(y_test, y_pred )

In [ ]:
y_score = knn_model.predict_proba(X_test)

metrics_knn = get_metrics(y_test, y_pred, y_score, name='KNeighborsClassifier')
metrics_knn

In [ ]:
first_row_df1 = metrics_lg.iloc[[0]]
first_row_df2 = metrics_rf.iloc[[0]]
first_row_df3 = metrics_cat.iloc[[0]]
first_row_df4 = metrics_knn.iloc[[0]]
first_row_df5 = metrics_lg_0.iloc[[0]]
first_row_df6 = metrics_rf_0.iloc[[0]]
first_row_df7 = metrics_cat_0.iloc[[0]]
first_row_df8 = metrics_knn_0.iloc[[0]]

first_rows_combined = pd.concat([first_row_df1, first_row_df2, first_row_df3, first_row_df4, first_row_df5, first_row_df6, first_row_df7, first_row_df8], axis=0)

remaining_rows_combined = pd.concat([metrics_lg.iloc[1:], metrics_rf.iloc[1:], metrics_cat.iloc[1:], metrics_knn.iloc[1:], metrics_lg_0.iloc[1:], metrics_rf_0.iloc[1:], metrics_cat_0.iloc[1:], metrics_knn_0.iloc[1:]], axis=0)

final_combined_df = pd.concat([first_rows_combined, remaining_rows_combined], axis=0).reset_index(drop=True)

final_combined_df = final_combined_df.sort_values(by='R2', ascending=False)
print(final_combined_df)

In [ ]:
first_row_df1 = metrics_lg.iloc[[0]]
first_row_df2 = metrics_rf.iloc[[0]]
first_row_df3 = metrics_cat.iloc[[0]]
first_row_df4 = metrics_knn.iloc[[0]]
first_row_df5 = metrics_lg_0.iloc[[0]]
first_row_df6 = metrics_rf_0.iloc[[0]]
first_row_df7 = metrics_cat_0.iloc[[0]]
first_row_df8 = metrics_knn_0.iloc[[0]]

first_rows_combined = pd.concat([first_row_df1, first_row_df2, first_row_df3, first_row_df4, first_row_df5, first_row_df6, first_row_df7, first_row_df8], axis=0)

remaining_rows_combined = pd.concat([metrics_lg.iloc[1:], metrics_rf.iloc[1:], metrics_cat.iloc[1:], metrics_knn.iloc[1:], metrics_lg_0.iloc[1:], metrics_rf_0.iloc[1:], metrics_cat_0.iloc[1:], metrics_knn_0.iloc[1:]], axis=0)

final_combined_df = pd.concat([first_rows_combined, remaining_rows_combined], axis=0).reset_index(drop=True)

final_combined_df = final_combined_df.sort_values(by='R2', ascending=False)
print(final_combined_df)

In [ ]:
y_pred = model_mlp.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy

In [ ]:
r2_score(y_test, y_pred )

In [ ]:
print(classification_report(y_test, y_pred))